Figure 2 – JSON data showing the sessions node and the first speaker node.  

In [13]:
USE [JsonTest];

DECLARE @AllJson AS NVARCHAR(MAX);

   SET @AllJson =
    (
        SELECT [Data] FROM [dbo].[JsonData]
    );

SELECT CASE
           WHEN ISJSON(@AllJson) = 1 THEN
               'Yippe Valid Json'
           ELSE
               'Boo hoo not Valid Json'
       END AS [CheckJSON];
 --if 1 returned then valid ON

Figure 3 - First query just to return data from the 'sessions' node

In [14]:
USE [JsonTest];

DECLARE @AllJson AS NVARCHAR(MAX);

SET @AllJson =
(
    SELECT [Data] FROM [dbo].[JsonData]
);

SELECT 
       [s].[Key]
     , [s].[Value]
     , [s].[Type]
     , JSON_VALUE(Value, '$.id')          AS SessionID
     , JSON_VALUE(Value, '$.title')       AS SessionTitle
     , JSON_VALUE(Value, '$.description') AS SessionDescription
     , JSON_VALUE(Value, '$.startsAt')    AS SessionStartsAt
     , JSON_VALUE(Value, '$.endsAt')      AS SessionEndsAt
     , JSON_VALUE(Value, '$.roomId')      AS RoomID
FROM OPENJSON(@AllJson, '$.sessions') AS s;

Figure 5 – Updated query to return the speakerID from the []speakers array.

In [15]:
USE [JsonTest];

DECLARE @AllJson AS NVARCHAR(MAX);

SET @AllJson =
(
    SELECT [Data] FROM [dbo].[JsonData]
);

SELECT
        JSON_VALUE(Value, '$.id')          AS SessionID
      , JSON_VALUE(Value, '$.title')       AS SessionTitle
      , JSON_VALUE(Value, '$.description') AS SessionDescription
      , JSON_VALUE(Value, '$.startsAt')    AS SessionStartsAt
      , JSON_VALUE(Value, '$.endsAt')      AS SessionEndsAt
      , JSON_VALUE(Value, '$.roomId')      AS RoomID
      , Speaker.[oi]                       AS SpeakerID
    FROM OPENJSON(@AllJson, '$.sessions')               s
        CROSS APPLY
    (SELECT Value FROM OPENJSON(s.Value, '$.speakers')) Speaker(oi)
    	ORDER BY [SessionID]

Figure 6 – TSQL query to return data from the []speakers array

In [16]:
USE [JsonTest];

DECLARE @AllJson AS NVARCHAR(MAX);

   SET @AllJson =
    (
        SELECT [Data] FROM [dbo].[JsonData]
    )
;

SELECT
        JSON_VALUE(Value, '$.id')             AS ID
      , JSON_VALUE(Value, '$.firstName')      AS FirstName
      , JSON_VALUE(Value, '$.lastName')       AS LastName
      , JSON_VALUE(Value, '$.bio')            AS SpeakerBio
      , JSON_VALUE(Value, '$.tagLine')        AS SpeakerTagLine
      , JSON_VALUE(Value, '$.profilePicture') AS ProfilePicture
	  , JSON_VALUE([LinksTw].[tw], '$.url')  AS TwitterHandle
    FROM OPENJSON(@AllJson, '$.speakers') s
        OUTER APPLY
    (   SELECT Value  FROM OPENJSON(s.Value, '$.links')
        WHERE Value LIKE '%Twitter%'
		) AS LinksTw(tw)
		;

Figure 7 – Result set with Session, Speaker and room details 

In [17]:
/*https://visakhm.blogspot.com/2016/07/whats-new-in-sql-2016-native-json_13.html*/
USE [JsonTest];

DECLARE @AllJson AS NVARCHAR(MAX);

   SET @AllJson =
    (
        SELECT [Data] FROM [dbo].[JsonData]
    )
;
WITH
[Sessions]
AS (SELECT
        --[s].[Key]
        JSON_VALUE(Value, '$.id')          AS SessionID
      , JSON_VALUE(Value, '$.title')       AS SessionTitle
      , JSON_VALUE(Value, '$.description') AS SessionDescription
      , JSON_VALUE(Value, '$.startsAt')    AS SessionStartsAt
      , JSON_VALUE(Value, '$.endsAt')      AS SessionEndsAt
      , JSON_VALUE(Value, '$.roomId')      AS RoomID
      , Speaker.[oi]                       AS SpeakerID
    FROM OPENJSON(@AllJson, '$.sessions')               s
        CROSS APPLY
    (SELECT Value FROM OPENJSON(s.Value, '$.speakers')) Speaker(oi)
--ORDER BY [SessionID]
)
,
[Speakers]
AS (SELECT
        JSON_VALUE(Value, '$.id')             AS ID
      , JSON_VALUE(Value, '$.firstName')      AS FirstName
      , JSON_VALUE(Value, '$.lastName')       AS LastName
      , JSON_VALUE(Value, '$.bio')            AS SpeakerBio
      , JSON_VALUE(Value, '$.tagLine')        AS SpeakerTagLine
      , JSON_VALUE(Value, '$.profilePicture') AS ProfilePicture
      , '@'
        + REPLACE(
                     SUBSTRING(
                                  JSON_VALUE([LinksTw].[tw], '$.url')
                                , LEN(JSON_VALUE([LinksTw].[tw], '$.url'))
                                  - CHARINDEX('/', REVERSE(JSON_VALUE([LinksTw].[tw], '$.url'))) + 2
                                , LEN(JSON_VALUE([LinksTw].[tw], '$.url'))
                                  - CHARINDEX('/', REVERSE(JSON_VALUE([LinksTw].[tw], '$.url')))
                              )
                   , '@'
                   , ''
                 )                            AS TwitterHandle
    FROM OPENJSON(@AllJson, '$.speakers') s
        OUTER APPLY
    (
        SELECT Value
        FROM OPENJSON(s.Value, '$.links')
        WHERE Value LIKE '%Twitter%'
    )                                     LinksTw(tw))
,
[Speakers_With_Sessions]
AS (SELECT [Sessions].[SessionID]
         , [Sessions].[SessionTitle]
         , [Sessions].[SessionDescription]
         , [Sessions].[SessionStartsAt]
         , [Sessions].[SessionEndsAt]
         , [Sessions].[RoomID]
         , [Sessions].[SpeakerID]
         , [Speakers].[FirstName]
         , [Speakers].[LastName]
         , [Speakers].[SpeakerBio]
         , [Speakers].[SpeakerTagLine]
         , [Speakers].[ProfilePicture]
         , [Speakers].[TwitterHandle]
    FROM [Sessions]
        LEFT JOIN [Speakers]
            ON [Speakers].[ID] = [Sessions].[SpeakerID])
,
[Rooms]
AS (SELECT JSON_VALUE(Value, '$.id')   AS RoomID
         , JSON_VALUE(Value, '$.name') AS RoomName
    FROM OPENJSON(@AllJson, '$.rooms') r)
SELECT [Speakers_With_Sessions].[SessionID]
     , [Speakers_With_Sessions].[SessionTitle]
     , [Speakers_With_Sessions].[SessionDescription]
     , [Speakers_With_Sessions].[SessionStartsAt]
     , [Speakers_With_Sessions].[SessionEndsAt]
     , [Speakers_With_Sessions].[RoomID]
     , [Speakers_With_Sessions].[SpeakerID]
     , [Speakers_With_Sessions].[FirstName]
     , [Speakers_With_Sessions].[LastName]
     , [Speakers_With_Sessions].[SpeakerBio]
     , [Speakers_With_Sessions].[SpeakerTagLine]
     , [Speakers_With_Sessions].[ProfilePicture]
     , [Speakers_With_Sessions].[TwitterHandle]
     , [Rooms].[RoomID]
     , [Rooms].[RoomName]
FROM [Speakers_With_Sessions]
    LEFT JOIN [Rooms]
        ON [Rooms].[RoomID] = [Speakers_With_Sessions].[RoomID];
